In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
train_data = pd.read_csv('persons_pics_train.csv')
predict_data = pd.read_csv('persons_pics_reserved.csv')

In [31]:
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']
X_predict = predict_data

PCA

In [32]:
# train the model PCA(svc_solver='full')
from sklearn.decomposition import PCA
pca = PCA(svd_solver='full')
pca.fit(X_train)

PCA(svd_solver='full')

In [33]:
explained_variances = pca.explained_variance_ratio_
cumulative_variances = np.cumsum(explained_variances)
num_components_needed = np.argmax(cumulative_variances > 0.99) + 1
print("The minimum number of components required for the level of explained variance to be greater than 0.95 is:", num_components_needed)


The minimum number of components required for the level of explained variance to be greater than 0.95 is: 465


In [34]:
pca = PCA(n_components=num_components_needed, svd_solver='full')
X_train_pca = pca.fit_transform(X_train)
X_predict_pca = pca.transform(X_predict)

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000], 'class_weight': [None, 'balanced'], 'random_state':[13]}]
cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3)
cv.fit(X_train_pca, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.797 total time=   0.5s
[CV 2/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.804 total time=   0.4s
[CV 3/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.796 total time=   0.3s
[CV 4/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.789 total time=   0.3s
[CV 5/5] END C=1, class_weight=None, gamma=0.001, kernel=linear, random_state=13;, score=0.792 total time=   0.3s
[CV 1/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.338 total time=   0.4s
[CV 2/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.340 total time=   0.4s
[CV 3/5] END C=1, class_weight=None, gamma=0.001, kernel=poly, random_state=13;, score=0.340 total time=   0.4s
[CV 4/5] END C=1, class_weight=N

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'class_weight': [None, 'balanced'],
                          'gamma': [0.001, 0.0001],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                          'random_state': [13]}],
             verbose=3)

In [36]:
# predict the labels of the test set
y_pred = cv.predict(X_predict_pca)

In [37]:
import csv
y_pred = pd.DataFrame(y_pred, columns=['label'])
y_pred.T.to_csv('reserved_SVM.csv', index=False, header=False, quotechar="'", quoting=csv.QUOTE_ALL)
